In [1]:
import importlib
import pandas as pd
import numpy as np
import re

from functions.get_mon_data import *
from functions.functions_dataframe import * 
from functions.functions_shared import *
from parameters.filters_lists import *
from parameters.route import *
from parameters.regs import *

pd.set_option("display.max_colwidth", 200)

In [2]:
"""
Завантажити базу закупівель ProZorro (деталі у README.txt)
"""
df_prozorro = pandas.read_csv('data/TrainingProZorroForSchoolProject20162019.csv', sep = ';', low_memory = False)

In [3]:
"""
Завантажити датафрейм МОН навчальниз закладів, підготовлений у "Fileter MON data.ipynb"
"""
df_mon_data_bez_upravlin_osvity = pandas.read_csv('data/df_bez_upravlin_osvity.csv', sep = ';')

In [4]:
"""
Створити список унікальних ЄДРПОУ освітніх закладів МОН
"""
mon_data_list = df_mon_data_bez_upravlin_osvity.edrpou.unique().tolist()

In [ ]:
"""  FOR DEBUGGING PURPOSE:

COLUMNS_TO_SHOW = []
VALUE_FILTER = ''
COLUMN_TO_FILTER = ''
debug_df = PUT_DARAFRAME_NAME_HERE[COLUMNS_TO_SHOW]
debug_df[debug_df[COLUMN_TO_FILTER] == VALUE_FILTER]

"""

### ШКОЛИ ЗАМОВНИКИ

##### Step 1: Створити базу навчальних закладів, які самостійно купують через ProZorro

In [24]:
"""
Відфільтрувати базу закупівель df_prozorro за колонкою IDOrganizator.
Фільтром є списко МОН з ЄДРПОУ начальних закладів mon_data_list (попередній крок)

df_tenders_by_schools - база закупівель, де навчальні заклади є замовниками
df_undefined_1 - база невизначених закупівель ProZorro
"""
df_tenders_by_schools = df_prozorro[df_prozorro['IDOrganizator'].isin(mon_data_list)]
df_undefined_1 = df_prozorro[~df_prozorro['IDOrganizator'].isin(mon_data_list)]

##### Step 2: Класифікація шкіл-замовників по типах за колонкою "Организатор"

In [25]:
df_tenders_by_schools_classified = classification_by_type_one_column(df_tenders_by_schools, ['Организатор'], filter_priority, filter_name_priority)

/home/artem/projects/tmp/school-procurements/functions/functions_dataframe.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  given_df['Тип закладу'] = "Невідомо"
/home/artem/projects/tmp/school-procurements/functions/functions_dataframe.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  given_df[column + "_check"] = ""
/home/artem/projects/tmp/school-procurements/functions/functions_dataframe.py:60: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  case = False, na = False, regex = True)
/home/a

##### Step 3: Нумерація шкіл-замовників

In [26]:
df_tenders_by_schools_numerated = numeration_one_column(df_tenders_by_schools_classified, ['Организатор'], reg_number)

/home/artem/projects/tmp/school-procurements/functions/functions_dataframe.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  given_df['Номер'] = given_df[column].str.findall(f"{regex}", flags = re.IGNORECASE).apply(''.join)


##### Step 4: Найменування шкіл-замовників

In [27]:
df_tenders_by_schools_named = naming_one_column(df_tenders_by_schools_numerated, ['Организатор'], reg_quotes)

/home/artem/projects/tmp/school-procurements/functions/functions_dataframe.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  given_df['Назва'] = given_df[column].str.findall(f"{regex}", flags = re.IGNORECASE).str[0].str[-1]


##### Step 5: Cтворення фінальної бази ШКІЛ ЗАМОВНИКІВ

In [28]:
"""
df_tenders_by_schools - база закупівель, де замовниками є навчальні заклади і де не зустрічаються ключові слова filter_poza
"""
df_tenders_by_schools = df_tenders_by_schools_named[~df_tenders_by_schools_named['Организатор'].astype(str).str.contains(r'({})'.format('|'.join(filter_poza)), case = False, na = False, regex = True)]

/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  after removing the cwd from sys.path.


In [29]:
"""
"Назва закладу" - колонка з фіналізованою назвою школи. Принцип визначення:
    1. Якщо колонка "Номер" не є пустою - додається до "Типу закладу"
    2. Якщо колонка "Номер" є пустою - "Назва закладу" отримує значення "Организатор"
    
* колонка "Назва" не враховується через високий ступінь повторюваності з "Организатор" і часткову точність
"""
df_tenders_by_schools['Номер'] = df_tenders_by_schools['Номер'].replace(r'^\s*$', np.nan, regex=True) # технічна дія: перетворити пусті строки в NaN
df_tenders_by_schools['Назва закладу'] = np.where(df_tenders_by_schools['Номер'].notnull(), 
                                                  df_tenders_by_schools['Тип закладу'] + ' ' + df_tenders_by_schools['Номер'], 
                                                  df_tenders_by_schools['Организатор'])

/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [31]:
"""
Створення колонки 'Заклад-замовник'
Видалення "технічних" колонок, створених функціями.
"""
df_tenders_by_schools['Заклад-замовник'] = True
col_names = df_prozorro.columns.tolist()
col_names += ['Заклад-замовник', 'Назва закладу']
df_tenders_by_schools = df_tenders_by_schools[col_names]

/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


### ШКОЛИ НЕ-ЗАМОВНИКИ

##### Step 5: Пошук закупівель шкіл-не-замовників за допомогою ключових слів

In [12]:
"""
З-поміж датафрейму невизначених закупівель (df_undefined_1) за ключовими словами у списках
filter_keywords, filter_keywords_strict відбираються закупівлі освітніх закладів у колонках: Тендер, ОписаниеТендера, Лот 

Примітка: регулярний вираз /b{}/b шукає за ключовими словами зі списку filter_keywords_strict_extended тільки 
ті паттерни, де ці ключові слова є окремими, а не зустрічають всередині слів. 
Це зменшить кількість "брудних" даних
"""
df_tenders_not_by_schools_by_keywords = df_undefined_1[df_undefined_1['Тендер'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords)), 
            case = False, na = False, regex = True)  | 
                                                      df_undefined_1['ОписаниеТендера'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords)), 
            case = False, na = False, regex = True) | 
                                                      df_undefined_1['Лот'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords)), 
            case = False, na = False, regex = True) | df_undefined_1['Тендер'].astype(str).str.contains(r'(/b{}/b)'.format('|'.join(filter_keywords_strict)), 
            case = False, na = False, regex = True) | df_undefined_1['ОписаниеТендера'].astype(str).str.contains(r'(/b{}/b)'.format('|'.join(filter_keywords_strict)), 
            case = False, na = False, regex = True) | df_undefined_1['Лот'].astype(str).str.contains(r'(/b{}/b)'.format('|'.join(filter_keywords_strict)), 
            case = False, na = False, regex = True)]

/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  app.launch_new_instance()
/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


##### Step 6: Класифікація шкіл-не-замовників за типами

In [13]:
df_tenders_not_by_schools_classified = classification_by_type_three_cols(df_tenders_not_by_schools_by_keywords,
                                                                         ['Тендер', 'ОписаниеТендера', 'Лот'], 
                                                                         filter_priority, 
                                                                         filter_name_priority)

/home/artem/projects/tmp/school-procurements/functions/functions_dataframe.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  given_df['Тип закладу'] = "Невідомо"
/home/artem/projects/tmp/school-procurements/functions/functions_dataframe.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  given_df[column + "_check"] = ""
/home/artem/projects/tmp/school-procurements/functions/functions_dataframe.py:40: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  case = False, na = False, regex = True)
/home/a

##### Step 7: Нумерація шкіл-не-замовників

In [56]:
"""
df_step7 - датафрейм, створений із закупівель df_tenders_not_by_schools_classified, де був успішно визначений "Тип закладу"
"""
df_step7 = df_tenders_not_by_schools_classified[df_tenders_not_by_schools_classified['Тип закладу'] != 'Невідомо'].copy()

In [57]:
"""
Функція long_numeration створює два дафайреми:
    df_tenders_not_by_schools_numerated - тендери, де номер визначився 
    df_step7_numerated_undefined - тендери, де номер не визначився
"""
df_tenders_not_by_schools_numerated, df_step7_numerated_undefined = long_numeration(df_step7, reg_number)
df_tenders_not_by_schools_numerated['Назва закладу'] = df_tenders_not_by_schools_numerated['Тип закладу'] + ' ' + df_tenders_not_by_schools_numerated['Номер']

##### Step 8: Найменування шкіл-не-замовників

In [58]:
"""
Завантажити датафрейм МОН власних назв навчальниз закладів, підготовлений у "Fileter MON data.ipynb"
school_names - список унікальних назв 
"""
df_names_clean = pandas.read_csv('data/df_names_clean.csv', sep = ';')
df_names_clean = df_names_clean[df_names_clean['Назва'].notnull()] # Видалити NaN
school_names = df_names_clean['Назва'].unique().tolist()

In [59]:
"""
Функція long_naming створює два дафайреми:
    df_tenders_not_by_schools_named - тендери, де номер визначився.
    df_tenders_not_by_schools_undefined - тендери шкіл не замовників, де не визначилась назва (і номер).
    Такі закупівлі вважаються закупівлями управінь освіти для різних закладів.
"""
df_tenders_not_by_schools_named, df_tenders_not_by_schools_undefined = long_naming(df_step7_numerated_undefined, reg_double_quotes, school_names)
df_tenders_not_by_schools_named['Назва закладу'] = df_tenders_not_by_schools_named['Тип закладу'] + ' ' + df_tenders_not_by_schools_named['Назва']

/home/artem/projects/tmp/school-procurements/functions/functions_dataframe.py:171: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  df_named['Назва'] = np.where(df_named['Тендер_naming_first'].astype(str).str.contains(r'^({})$'.format('|'.join(school_names)), case = False, na = False, regex = True), df_named['Тендер_naming_first'], "Невідомо")
/home/artem/projects/tmp/school-procurements/functions/functions_dataframe.py:175: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  df_named_undefined_1['Назва'] = np.where(df_named_undefined_1['Лот_naming_first'].astype(str).str.contains(r'^({})$'.format('|'.join(school_names)), case = False, na = False, regex = True), df_named_undefined_1['Лот_naming_first'], "Невідомо")
/home/artem/projects/tmp/school-procurements/functions/functions_dataframe.py:175: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

##### Step 9: Об'єднання двох датафреймів шкіл-не-замовників

In [60]:
"""
df_tenders_not_by_schools - датафрейм шкіл-не-замовників з визначеними номерами і назвами
Створення колонки 'Заклад-замовник'
"""
frames = [df_tenders_not_by_schools_numerated, df_tenders_not_by_schools_named]
df_tenders_not_by_schools = pd.concat(frames)
df_tenders_not_by_schools['Заклад-замовник'] = False
df_tenders_not_by_schools = df_tenders_not_by_schools[col_names]

### ВСІ ШКОЛИ

##### Step 10: Об'єднання датафреймів шкіл-не-замовників та шкіл-замовників

In [62]:
frames = [df_tenders_by_schools, df_tenders_not_by_schools]
df_tenders = pd.concat(frames)

In [64]:
df_tenders.to_csv('data/df_tenders.csv', sep=';', index=False)